<h2><center>ECE 5760: Advanced Microcontroller Design and System-on-Chip</center></h2>
<h1><center>Final Project: Fourier Drawing</center></h2>
<h3><center>Yibo Yang (), Brian Hsu (), Jasmin An (ja499)</center></h3>

<h1>1. Introduction</h1>
<p>In this project, we aim to use the Fourier analysis and synthesis to redraw a user-input image on the VGA and play the sound of the image. The Fourier analysis and synthesis derive a harmonic wave equation that approximates the user-input image when its coordinates are given in sequence. Once the equation is calculated, the system redraws the image on the VGA and outputs the sound of the image. Also, we provide various features, such as redrawing the next image, morphing the current image to another image, and changing the amplitude of a specific frequency in the harmonic equation. We provide the VGA, mouse, and SSH terminal as the user interface and output the reproduced image on the same VGA and the sound using the speaker. We first started by implementing the Fourier analysis and synthesis algorithms in C and setting up the entire system, including the audio system and user interface as shown in Figure 1. Then we implemented the synthesis algorithm on the FPGA to accelerate it using parallelism.</p>

<figure>
<center><img src="img/front_page_img.jpeg" width="800" height="600" align="center"/></center>
<figcaption align="center"><b>Fig.1. Overall system.</b></figcaption>
</figure>

<h1>2. High Level Design</h1>
<h2>2.1. Rationale</h2>
<p>We are inspired by this video [1]. The math involved with the Fourier drawing has repetitiveness, which presents some possibility of parallelism and modularity. We believed that implementing these on the FPGA would make good use of the FPGA. Also, we included several user interfaces to make the user experience better. We added the audio feature because we thought it could be interesting to listen to what an image sounds like and to experiment with the audio system.</p>
<h2>2.2. Background Math: Fourier Analysis and Synthesis</h2>
<h2>2.3. Hardware/Software Tradeoffs</h2>
<p>We initially implemented both the analysis and synthesis portions of the algorithm on the host using C. We integrated the user interface components – mouse and audio system – to have a working baseline design. However, the software-only design could not meet the timing constraint of the audio system, which will be discussed in greater detail in a later part of the report. The synthesis portion of the algorithm described above is largely composed of independent calculations, except for the last summation. Therefore, we parallelized the independent calculations on the FPGA. Also, we used simple lookup tables for computing the sine and cosine functions, which will help accelerate the calculation. However, implementation of each module that calculates one harmonic, phasor.sv, in the synthesis portion requires four multipliers. We use 4.23 fixed-point and there are two multiplications required for sine and cosine and their corresponding coefficients. We calculate the x and y coordinates separately, so there are four multipliers required to calculate for one harmonic. Since there are 82 multipliers, we can use up to 21 harmonics to represent the input image, which is controlled by th_harmonics in vga_to_FPGA_audio.c.</p>

<h1>3. Program/hardware design</h1>
<h2>3.1. Program Details</h2>
<h3>3.1.1. Mouse</h3>
<p>We used the reference code given in [2] to enable the mouse input. We included a debouncing logic in order to count only one input when the button was pressed and held. This is important for our application since the number of clicks are set at compile time and their coordinates need to be sent in the exactly the same sequence for the Fourier analysis. Since the analysis requires the input image to be a closed loop, we automatically add a last point, which is a copy of the first point. Also, we added the drawing logic, which draws a line through the previous click to the current click to show the overall shape that the user is drawing.</p>
<h3>3.1.2. Analysis</h3>
<h3>3.1.3. PIO</h3>
<p>The coefficients calculated by the analysis function on the host are sent to the FPGA for parallel synthesis via PIO ports. We define seven pio ports, i.e. pio_a, pio_b, pio_c, pio_d, pio_reset, pio_wr_en, and pio_wr_addr. We send the coefficients of the sine terms for the x coordinate via pio_a and the cosine terms via pio_b and similarly for pio_c and pio_d. Since we need to send as many coefficients as the number of harmonics for each of pio_a, pio_b, pio_c, and pio_d, we communicate with the FPGA using pio_reset, pio_wr_addr, and pio_wr_en. We notify that a new sequence of coefficients will be sent using pio_reset. The pio_wr_addr indicates the index of the coefficients in the sequence, and pio_wr_en is set for at least a cycle on the FPGA to notify the FPGA to read in the value. The FPGA sets the registers that saves the values from the pio ports to zero when the pio_reset is high. Otherwise, when the pio_wr_en is high, it will be saving the values from the pio_a, pio_b, pio_c, and pio_d ports to their respective registers. We noticed that sometimes the raw values are too small that we cannot hear well when output to the speaker. The SW[7:4] are used to control the loudness, which corresponds to 2^5 ~ 2^8 times the original loudness. </p>
<h3>3.1.4. Draw</h3>
<h2>3.2. Hardware Details</h2>
<h3>3.2.1. Audio</h3>
<h3>3.2.2. Synthesis</h3> 
<h2>3.3. Reference Codes</h2>
<p>We did not use any code that is involved with existing patents, copyrights, and trademarks.</p>
<h2>3.4. Trial and Error</h2>

<h1>4. Results of the Design</h1>
<figure>
<table align="center">
  <tr>
    <th>Input image (red), reproduced image (yellow)</th>
    <th>Analysis (uSec)</th>
    <th>Synthesis + print to VGA (uSec)</th>
  </tr>
  <tr>
    <td><img src="img/square.jpeg" width="300" height="300" align="center"/></td>
    <td>203~239</td>
    <td>1700~2900</td>
  </tr>
  <tr>
    <td><img src="img/circle.jpeg" width="300" height="300" align="center"/></td>
    <td>459~501</td>
    <td>2261</td>
  </tr>
  <tr>
    <td><img src="img/star.jpeg" width="300" height="300" align="center"/></td>
    <td>383~634</td>
    <td>3299</td>
  </tr>
  <tr>
    <td><img src="img/complicated.jpeg" width="300" height="300" align="center"/></td>
    <td>1167~1351</td>
    <td>10863</td>
  </tr>
  <figcaption align="center"><b>Table 1. Timing of the analysis and synthesis done on host for different kinds of input image. Note that 21 harmonics are used.</b></figcaption>
</table>
</figure>

Note that synthesis is computed on both the host and FPGA because the audio system requires that the next value to output to the DAC is ready at 48 kHz, which gives approximately 20 uSec until the next value, whereas the VGA allows a lower rate of update. In order to meet this timing constraint, we implemented the synthesis part on the FPGA just for the audio system, i.e. synthesis is implemented both on the host for the VGA and in the FPGA for the audio. The synthesis state machine runs at 50 MHz, and using parallelism all the computations are finished within one cycle. Therefore, it is less than 0.02 uSec per synthesis on the FPGA, which is within the timing requirements for the audio system. In Table 1, we only have timings that combine the synthesis and output to VGA because it was difficult for us to measure without the VGA output. Despite this, the values are significantly larger than the audio system constraint, indicating that the synthesis implementation on the FPGA is much faster than that on the host. 


For our project, we define accuracy as how accurately the system redraws the input image. The first column in Table 1 shows that the accuracy decreases as the shape of the user input has increasing complexity and more sharp edges. This is expected as the Fourier drawing requires higher frequencies, or more harmonics, to successfully represent the details, as shown by the x and y spectrums in the lower left corners of the images. 

In terms of usability, we provide several features that we found to be useful when we were testing our system. Table 2 is the list of the user interface features that our system provides. Also, we included some interesting visual features. For instance, when we morph the image, we show the progress of morphing both on the VGA and sound, as shown in Figure 2. 

<figure>
<table align="center">
  <tr>
    <th>Feature</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>Mouse left click</td>
    <td>To input user image on the VGA; the number of clicks can be changed in the host code; automatically makes a closed image</td>
  </tr>
  <tr>
    <td>Mouse right click</td>
    <td>To stop clicking before the number of clicks is reached; automatically makes a closed image</td>
  </tr>
  <tr>
    <td>Redraw input image</td>
    <td>‘r’ in SSH terminal</td>
  </tr>
  <tr>
    <td>Morph current image to a different image</td>
    <td>‘m’ in SSH terminal</td>
  </tr>
  <tr>
    <td>Change the amplitude of a harmonic</td>
    <td>‘c’ in SSH terminal</td>
  </tr>
  <figcaption align="center"><b>Table 2. Table for features</b></figcaption>
</table>
</figure>

<figure>
<center><img src="img/morph0.jpeg" width="300" height="300" align="center"/></center>
<center><img src="img/morph1.jpeg" width="300" height="300" align="center"/></center>
<center><img src="img/morph2.jpeg" width="300" height="300" align="center"/></center>
<center><img src="img/morph3.jpeg" width="300" height="300" align="center"/></center>
<center><img src="img/morph4.jpeg" width="300" height="300" align="center"/></center>
<center><img src="img/morph5.jpeg" width="300" height="300" align="center"/></center>
<figcaption align="center"><b>Fig.2. Morphing.</b></figcaption>
</figure>